In [271]:
import requests  
from bs4 import BeautifulSoup

In [278]:
# загрузим полезные модули
from collections import defaultdict as dd

# из pandas берем удобные функции для таблиц. можно не использовать её
import pandas as pd

import matplotlib.pyplot as plt

# визуализация циклов
from tqdm.notebook import tqdm

import time
import os
from fake_useragent import UserAgent

__Выгружаем названия фильмов__

In [272]:
# подключаемся к сайту
url = 'https://www.ivi.ru/movies/all?ysclid=lh3kbxx7q6794249776'
response = requests.get(url)
response

<Response [200]>

In [273]:
tree = BeautifulSoup(response.content, 'html.parser') #получаем структуру сайта в питоне

In [275]:
#вынимаем из кода название фильма
books_on_one_page=tree.find_all('span','nbl-slimPosterBlock__titleText')
books_on_one_page[0] #с первой страницы

<span class="nbl-slimPosterBlock__titleText">На солнце, вдоль рядов кукурузы</span>

In [276]:
list_movies=[] #создаем список наших фильмов с первой страницы, чтобы убедиться, что все работает 
for index in range(len(books_on_one_page)):
    elem_soup = BeautifulSoup(str(books_on_one_page[index]), 'html.parser')
    title = elem_soup.find('span','nbl-slimPosterBlock__titleText').text
    list_movies.append(title)

In [277]:
list_movies #интересный факт: порядок фильмов на сайте часто меняется

['На солнце, вдоль рядов кукурузы',
 'Подрыв',
 'Беспринципные в деревне',
 'Нюрнберг',
 'Моё прекрасное несчастье',
 'Взломщик',
 'Любовь-морковь: Восстание машин',
 'Эскортница',
 'Призрачный дом',
 'Ложная память',
 'Операция «Фортуна»: Искусство побеждать',
 'Праведник',
 'Сокровища партизанского леса',
 'Аве Мария',
 'Моя пиратская свадьба',
 'Непослушная',
 'Горизонт в огне',
 'Свободные отношения',
 'Т-34. Полная версия',
 'Рыцарь теней',
 'При чём тут любовь?',
 'Профиль',
 'Здоровый человек',
 'Мой любимый призрак',
 'Девятаев',
 'Чёрная метка',
 'Крушение',
 '13 минут',
 'Вакцина от коррупции (на казахском языке с русскими субтитрами)']

In [279]:
#проделываем эти же операции для следующих страниц

for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page = tree.find_all('span','nbl-slimPosterBlock__titleText')
    for index in range(len(books_on_one_page)):
        elem_soup = BeautifulSoup(str(books_on_one_page[index]), 'html.parser')
        title = elem_soup.find('span','nbl-slimPosterBlock__titleText').text
        list_movies.append(title)
        

In [280]:
len(list_movies) #количество собранных фильмов

2291

__Выгружаем рейтинг фильмов__

In [281]:
#вновь подключаемся к сайту (боялись, что может слететь)

url2 = 'https://www.ivi.ru/movies/all?ysclid=lh3kbxx7q6794249776'
response2 = requests.get(url2)
response2

<Response [200]>

In [282]:
tree2 = BeautifulSoup(response2.content, 'html.parser')

In [283]:
#смотрим в какой части кода сайта находится рейтинг фильмов

books_on_one_page_rate=tree2.find_all('div','nbl-ratingCompact__value')
books_on_one_page_rate[0]

<div class="nbl-ratingCompact__value"><div class="nbl-ratingCompact__valueInteger">8</div><div class="nbl-ratingCompact__valueFraction">,<!-- -->4</div></div>

In [284]:
#создаем список этих фильмов

rate_movies=[]
for index in range(len(books_on_one_page_rate)):
    elem_soup_rate = BeautifulSoup(str(books_on_one_page_rate[index]), 'html.parser')
    ratio = elem_soup_rate.find('div','nbl-ratingCompact__value').text
    rate_movies.append(ratio)

In [285]:
#повторяем эту операцию для следующих страниц

for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree3 = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_2 = tree3.find_all('div','nbl-ratingCompact__value')
    for index in range(len(books_on_one_page_2)):
        elem_soup_2 = BeautifulSoup(str(books_on_one_page_2[index]), 'html.parser')
        ratio = elem_soup_2.find('div','nbl-ratingCompact__value').text
        rate_movies.append(ratio)

In [286]:
len(rate_movies) #количество полученных рейтингов совпала с количество названий

2291

__Выгружаем год выпуска фильма__

In [287]:
url3 = 'https://www.ivi.ru/movies/all?ysclid=lh3kbxx7q6794249776'
response3 = requests.get(url3)
response3

<Response [200]>

In [288]:
tree3 = BeautifulSoup(response3.content, 'html.parser')

In [289]:
#смотрим в какой части кода сайта находится год выпуска

books_on_one_page_years=tree3.find_all('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow') 
books_on_one_page_years[0]

<div class="nbl-poster__propertiesInfo"><div class="nbl-poster__propertiesRow">2023, Россия, Драмы</div><div class="nbl-poster__propertiesRow">89 минут</div></div>

In [290]:
#создаем список с годами с первой страницы, отделив их от страны и жанра

year_movies=[]
for index in range(len(books_on_one_page_years)):
    elem_soup_year = BeautifulSoup(str(books_on_one_page_years[index]), 'html.parser')
    year = elem_soup_year.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[0]
    year_movies.append(year)

In [291]:
#для 80 страниц

for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree4 = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_3 = tree4.find_all('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow')
    for index in range(len(books_on_one_page_3)):
        elem_soup_3 = BeautifulSoup(str(books_on_one_page_3[index]), 'html.parser')
        year = elem_soup_3.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[0]
        year_movies.append(year)

In [292]:
len(year_movies) #собрали необходимое количество наблюдений

2291

__Выгружаем страну__

In [293]:
#из той же строки кода достаем страну фильма, отделив от года и жанра

country_movies=[]
for index in range(len(books_on_one_page_years)):
    elem_soup_country = BeautifulSoup(str(books_on_one_page_years[index]), 'html.parser')
    country = elem_soup_country.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[1]
    country_movies.append(country)

In [294]:
#для следующих страниц

for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree5 = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_4 = tree5.find_all('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow')
    for index in range(len(books_on_one_page_4)):
        elem_soup_4 = BeautifulSoup(str(books_on_one_page_4[index]), 'html.parser')
        country = elem_soup_4.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[1]
        country_movies.append(country)

In [295]:
len(country_movies) #собрали необходимое количество наблюдений

2291

__Выгружаем жанры__

In [296]:
url4 = 'https://www.ivi.ru/movies/all?ysclid=lh3kbxx7q6794249776'
response4 = requests.get(url4)
response4

<Response [200]>

In [297]:
tree4 = BeautifulSoup(response4.content, 'html.parser')

In [298]:
#жанры выгружаются вместе с продолжительностью фильмов 
# и имеют формат str

item4=tree4.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow') 
type(item4.text.split(', ')[2].split(' ')[0])

str

In [299]:
#разделяем жанр от продолжительности с помощью регуляризации
#(по совету ассистента нашли информацию в интернете)

import re 
res = [re.findall(r'(\w+?)(\d+)', item4.text.split(', ')[2].split(' ')[0])[0]] #https://translated.turbopages.org/proxy_u/en-ru.ru.bc230ef0-644ebcdd-a60c0a1a-74722d776562/https/www.geeksforgeeks.org/python-splitting-text-and-number-in-string/
res[0][0] #https://docs-python.ru/standart-library/modul-re-python/funktsija-findall-modulja-re/

'Драмы'

In [300]:
#Убеждаемся, что наш код работает для одной страницы
#создаем список жанров

genre_movies=[]
for index in range(len(books_on_one_page_years)):
    elem_soup_genre = BeautifulSoup(str(books_on_one_page_years[index]), 'html.parser')
    genre_minute = elem_soup_genre.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[2].split(' ')[0]
    seperate_genre= [re.findall(r'(\w+?)(\d+)',genre_minute)[0]][0][0]
    genre_movies.append(seperate_genre)

In [301]:
len(genre_movies) #количество жанров с первой страницы

29

In [302]:
#собираем данные со следующих страниц
#Обратим внимае, что есть жанр "Для детей", который пишется в два слова
#и в одном наблюдении пропущена страна, что мешало собрать данные прошлым кодом

pages_last=0
for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree6 = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_5 = tree6.find_all('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow')
    for index in range(len(books_on_one_page_5)):
        elem_soup_5 = BeautifulSoup(str(books_on_one_page_5[index]), 'html.parser')
        genre_minute1 = elem_soup_5.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')
        if len(genre_minute1)==3:
            gf= genre_minute1[2]
            if len(gf.split(' '))==2: #для детей жанр
                tyu= gf.split(' ')[0]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][0]
                genre_movies.append(s)
            else:
                lp= gf.split(' ')[1]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][0]
                genre_movies.append(f"{gf.split(' ')[0]} {s}")
        else: #Неистовый где пропуск страны
            gf=genre_minute1[1]
            if len(gf.split(' '))==2:
                tyu= gf.split(' ')[0]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][0]
                genre_movies.append(s)
            else:
                tyu= gf.split(' ')[1]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][0]
                genre_movies.append(f"{gf.split(' ')[0]} {s}")
    pages_last+=1

In [303]:
pages_last

78

In [304]:
len(genre_movies) #собрали необходимосе количество жанров

2291

__Выгружаем продолжительность__

In [305]:
#переходим к продолжительности
#вспомним, что она выгружается вместе с жанром (поэтому вновь используем регуляризацию)

minute_movies=[]
for index in range(len(books_on_one_page_years)):
    elem_soup_genre = BeautifulSoup(str(books_on_one_page_years[index]), 'html.parser')
    genre_minute = elem_soup_genre.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')[2].split(' ')[0]
    seperate_genre= [re.findall(r'(\w+?)(\d+)',genre_minute)[0]][0][1]
    minute_movies.append(seperate_genre)

In [306]:
minute_movies[:2] #сверяем первые два фильма, чтобы они шли в правильном порядке

['89', '110']

In [307]:
# учитываем все предыдущие комментарии и обращаем внимание на жанр "Для детей" и фильм с пропущенной страной 
pages_last=0
for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree6 = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_5 = tree6.find_all('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow')
    for index in range(len(books_on_one_page_5)):
        elem_soup_5 = BeautifulSoup(str(books_on_one_page_5[index]), 'html.parser')
        genre_minute1 = elem_soup_5.find('div', 'nbl-poster__propertiesInfo','nbl-poster__propertiesRow').text.split(', ')
        if len(genre_minute1)==3:
            gf= genre_minute1[2]
            if len(gf.split(' '))==2: #для детей жанр
                tyu= gf.split(' ')[0]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][1]
                minute_movies.append(s)
            else:
                lp= gf.split(' ')[1]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][1]
                minute_movies.append(f"{s}")
        else: #Неистовый где пропуск страны
            gf=genre_minute1[1]
            if len(gf.split(' '))==2:
                tyu= gf.split(' ')[0]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][1]
                minute_movies.append(s)
            else:
                tyu= gf.split(' ')[1]
                s= [re.findall(r'(\w+?)(\d+)',gf)[0]][0][1]
                minute_movies.append(f"{s}")
    pages_last+=1

In [308]:
pages_last

78

In [309]:
minute_movies[270:280] #сверим, что мы верно спарсили данные

['115', '105', '75', '106', '131', '92', '93', '112', '105', '104']

In [310]:
len(minute_movies) #получили необходимое количество наблюдений

2291

__Выгружаем возрастное ограничение__

In [311]:
#далее переходим к выгрузке возрастного ограничения
url5 = 'https://www.ivi.ru/movies/all?ysclid=lh3kbxx7q6794249776'
response5 = requests.get(url5)
response5

<Response [200]>

In [312]:
tree7 = BeautifulSoup(response5.content, 'html.parser')

In [313]:
#определяем, где в коде сайта находится ограничение
books_on_one_page_years=tree7.find_all('div', "nbl-poster nbl-poster_type_poster nbl-poster_extrasMode_rusk nbl-slimPosterBlock__nbl-poster") 
books_on_one_page_years[0]
#в последней строчке данного блока можно заметить nbl-ageBadge_value_6 и это то, что нам надо

<div class="nbl-poster nbl-poster_type_poster nbl-poster_extrasMode_rusk nbl-slimPosterBlock__nbl-poster"><div class="nbl-poster__imageWrapper"><img alt="На солнце, вдоль рядов кукурузы" class="nbl-poster__image" data-stub="false" src="https://thumbs.dfs.ivi.ru/storage5/contents/9/f/19a5fa548a9b111455b04f3d54d76a.jpg/234x360/?q=85"/></div><div class="nbl-poster__properties"><div class="nbl-poster__hoards"><div class="hoard hoard_type_favorite hoard_style_kaera hoard_iconOnly nbl-poster__hoard"><div class="hoard__icon"></div></div><div class="hoard hoard_type_similar hoard_style_kaera hoard_iconOnly nbl-poster__hoard"><div class="hoard__icon"></div></div><div class="hoard hoard_type_rate hoard_style_kaera hoard_iconOnly nbl-poster__hoard"><div class="nbl-icon nbl-icon_rating_20 hoard__nbl-icon hoard__icon" style="width:20px;height:20px;font-size:20px;line-height:20px"></div></div><div class="hoard hoard_type_dislike hoard_style_dislike hoard_iconOnly nbl-poster__hoard"><div class="hoard

In [314]:
#добираемся до ограничения многократно сплитуя элементы этого блока
# проверяем работу для первой страницы

restriction=[]
for i in range(len(books_on_one_page_years)):
    film=books_on_one_page_years[i]
    splitted=film.prettify().split('\n')
    lenght=len(splitted)
    restrict=splitted[lenght-3].split(' ')[4].split('_')
    long=len(restrict)
    limit=restrict[long-1]
    restriction.append(limit)
len(restriction)

29

In [315]:
#проделываем для следующих страниц

for page in range(2,80):
    url = f'https://www.ivi.ru/movies/all/page{page}?ysclid=lh3kbxx7q6794249776'
    response = requests.get(url)
    tree_limit = BeautifulSoup(response.content, 'html.parser')
    books_on_one_page_limit = tree_limit.find_all('div', "nbl-poster nbl-poster_type_poster nbl-poster_extrasMode_rusk nbl-slimPosterBlock__nbl-poster")
    for i in range(len(books_on_one_page_limit)):
        film=books_on_one_page_limit[i]
        splitted=film.prettify().split('\n')
        lenght=len(splitted)
        restrict=splitted[lenght-3].split(' ')[4].split('_')
        long=len(restrict)
        limit=restrict[long-1]
        restriction.append(limit)

In [316]:
len(restriction) #получаем необходимое количество наблюдений

2291

In [317]:
#наконец-то создаем DataFrame с выгруженными данными

df = pd.DataFrame({'Название': list_movies,'Страна': country_movies,
                   'Жанр':genre_movies,'Год выпуска': year_movies,
                   'Возрастное ограничение':restriction,'Продолжительность':minute_movies, 
                   'Рейтинг':rate_movies})

In [318]:
#количество данных каждого признака одинаковое
print( len (list_movies), len (country_movies), len (year_movies),len (genre_movies),len (minute_movies),len (rate_movies))

2291 2291 2291 2291 2291 2291


In [319]:
df

,Название,Страна,Жанр,Год выпуска,Возрастное ограничение,Продолжительность,Рейтинг
0,"На солнце, вдоль рядов кукурузы",Россия,Драмы,2023,6,89,"8,4"
1,Подрыв,Беларусь,Драмы,2022,16,110,"7,5"
2,Беспринципные в деревне,Россия,Комедии,2023,18,86,"7,2"
3,Нюрнберг,Россия,Драмы,2023,12,124,"8,1"
4,Моё прекрасное несчастье,США,Комедии,2023,18,91,"7,7"
...,...,...,...,...,...,...,...
2286,Русалка. Озеро мертвых,Россия,Ужасы,2018,16,83,"5,5"
2287,Форсаж 4,США,Криминал,2009,16,106,"8,8"
2288,Полночь на злаковом поле,США,Триллеры,2021,16,98,"6,7"
2289,Грань времени,США,Фантастика,2016,16,99,"5,2"


In [320]:
#запишем в файл
df.to_csv('project2.csv', index=False, columns=['Название', 'Страна', 'Жанр', 'Год выпуска', 'Возрастное ограничение', 'Продолжительность','Рейтинг'])